# Convert the model to ONNX runtime format

Install required dependencies

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_c

Convert PT model to ONNX

In [3]:
import torch
from transformers import DPRReader, DPRReaderTokenizer

model_id = "facebook/dpr-reader-multiset-base"
tokenizer = DPRReaderTokenizer.from_pretrained(model_id)
model = DPRReader.from_pretrained(model_id)

model.eval()

# Prepare from dummy input
dummy_inputs = tokenizer(
    questions=["What is love?"],
    titles=["Haddaway"],
    texts=["'What Is Love' is a song recorded by the artist Haddaway"],
    return_tensors="pt",
    padding=True,
    truncation=True,
)

inputs_onnx = tuple(dummy_inputs[k] for k in ['input_ids', 'attention_mask'])

#Export the model to ONNX
torch.onnx.export(
    model,
    inputs_onnx,
    "models/facebook-dpr.onnx",
    opset_version=12,
    input_names=["input_ids", "attention_mask"],
    output_names=["start_logits", "end_logits"],
    dynamic_axes = {
        "input_ids": {0: 'batch_size', 1: 'sequence'},
        "attention_mask": {0: 'batch_size', 1: 'sequence'},
        "start_logits": {0: 'batch_size', 1: 'sequence'},
        "end_logits": {0: 'batch_size', 1: 'sequence'}
    },
    do_constant_folding=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRReaderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-reader-multiset-base were not used when initializing DPRReader: ['span_predictor.encoder.bert_model.pooler.dense.bias', 'span_predictor.encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRReader from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRReader from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: 'models/facebook-dpr.onnx'